In [3]:
import pandas as pd
from funciones_recommendation_system import *
from funciones_filters_recommendation_system import *
from funciones_match import *
from funciones_plot_validation import *

In [5]:
def convert_to_str(val):
    if isinstance(val, float):
        return str(int(val))  # Elimina el .0 convirtiendo a int primero
    return str(val)  # Deja los strings como están

In [30]:
path = '/export/data_ml4ds/AI4U/Datasets/'
path_sim_matrices = '/export/data_ml4ds/AI4U/Datasets/similarity_matrices/publications/{}_sim_matrix.parquet'

version_wp = '20240510'
version_rp = '20240321'

agg_methods = ['sum', 'mean', 'mean_imp']
methods = ['BERT', 'bhattacharyya', 'separated', 'semiseparated']

df_publications, df_projects, df_publications_researchers, df_projects_researchers, df_researchers, df_calls = get_datasets(path, version_wp, version_rp)
df_project_publication_researcher = pd.concat([df_publications_researchers[['id_paper', 'id_researcher']], df_projects_researchers[['actID', 'id_researcher']].rename(columns={'actID':'id_paper'})], ignore_index=True)
df_project_publication_researcher['id_paper'] = df_project_publication_researcher['id_paper'].apply(convert_to_str)
df_project_publication_researcher['id_researcher'] = df_project_publication_researcher['id_researcher'].astype(str)

df_val = pd.read_excel('/export/usuarios_ml4ds/mafuello/Github/recommendation_system_validation/validation_set.xlsx')
df_val['id_researcher'] = df_val['id_researcher'].astype(str)

In [75]:
path_sim_matrices = '/export/data_ml4ds/AI4U/Datasets/similarity_matrices/publications/{}_sim_matrix.parquet'
path_save = '/export/data_ml4ds/AI4U/Datasets/similarity_matrices/researchers_filtered_{}/similarity_{}_{}.parquet'

In [84]:
method = methods[3]
sim_matrix_publis = pd.read_parquet(path_sim_matrices.format(method))

In [85]:
# filter number of publis
df_project_publication_researcher_filtered = filter_by_num_publis(39, df_project_publication_researcher, df_researchers)
similarity_researchers_sum = agg_sum(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)
similarity_researchers_mean = agg_mean(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)
similarity_researchers_mean_imp = agg_mean_imp(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)

Processing researchers IDs: 100%|██████████| 528/528 [01:00<00:00,  8.77 id_RP/s]


In [86]:
similarity_researchers_sum.to_parquet(path_save.format('num_publis', method, 'sum'))
similarity_researchers_mean.to_parquet(path_save.format('num_publis', method, 'mean'))
similarity_researchers_mean_imp.to_parquet(path_save.format('num_publis', method, 'mean_imp'))

In [89]:
# filter number of IPs
for method in methods:
    sim_matrix_publis = pd.read_parquet(path_sim_matrices.format(method))

    # filter number of publis
    df_project_publication_researcher_filtered = filter_by_num_ip(3, df_project_publication_researcher, df_researchers)
    similarity_researchers_sum = agg_sum(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)
    similarity_researchers_mean = agg_mean(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)
    similarity_researchers_mean_imp = agg_mean_imp(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)

    similarity_researchers_sum.to_parquet(path_save.format('num_IP', method, 'sum'))
    similarity_researchers_mean.to_parquet(path_save.format('num_IP', method, 'mean'))
    similarity_researchers_mean_imp.to_parquet(path_save.format('num_IP', method, 'mean_imp'))

Processing researchers IDs: 100%|██████████| 490/490 [00:54<00:00,  8.95 id_RP/s]


In [91]:
# filter number of IPs and number of publis
method = methods[3]

for method in methods:
    sim_matrix_publis = pd.read_parquet(path_sim_matrices.format(method))

    # filter number of publis
    df_project_publication_researcher_filtered = filter_by_num_publis(39, df_project_publication_researcher, df_researchers)
    df_project_publication_researcher_filtered = filter_by_num_ip(3, df_project_publication_researcher_filtered, df_researchers)
    similarity_researchers_sum = agg_sum(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)
    similarity_researchers_mean = agg_mean(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)
    similarity_researchers_mean_imp = agg_mean_imp(sim_matrix_publis, df_calls, df_project_publication_researcher_filtered, df_researchers)

    similarity_researchers_sum.to_parquet(path_save.format('combined', method, 'sum'))
    similarity_researchers_mean.to_parquet(path_save.format('combined', method, 'mean'))
    similarity_researchers_mean_imp.to_parquet(path_save.format('combined', method, 'mean_imp'))

Processing researchers IDs: 100%|██████████| 350/350 [00:40<00:00,  8.73 id_RP/s]
